**Предисловие:**

**Реализация основана на работе:** ...

**Результаты данной работы представлены в статье (при использовании просьба ссылаться на следующую работу):** ...

**Выборки данных:**
>*    Подписи к изображениям на английском языке из выборки COCO Image Captions
*    Стихи на русском языке с сайта stihi.ru

**Описание работы:**
> В данной работе проведены реализация, обучение и тестирование нейронных сетей для генерации случайных коротких текстов. Используются такие нейронные сети как LSTM и SeqGAN. Проведена оценка качества генерации текста на основе метрики BLEU. Для обучения и тестирования используются следующие выборки данных:
*   Подписи к изображениям на английском языке из выборки COCO Image Captions
*   Стихи на русском языке с сайта stihi.ru




Список всех установленных библиотек и их версии

In [1]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

Подключение к Google Drive

In [2]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

path = '/content/drive/My Drive/SeqGAN/'
os.chdir(path)

Объявление параметров по умолчанию

In [4]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys

import torch
import torch.optim as optim
import torch.nn as nn

import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.init as init

import helpers # перетащить функции в код

DATASET = 'coco_image_captions' # название выборки данных список выборок: ['stihi_ru', 'coco_image_captions']
CUDA = True # использование cuda (gpu/tpu): True/False
BATCH_SIZE = 1000 # количество примеров в партии/батче
MLE_TRAIN_EPOCHS = 30 # количество эпох при обучении генератора на основе MLE
DIS_TRAIN_ITERATIONS = 300 # количество итераций при обучении дискриминатора (итерации содержит несколько эпох, на каждой итерации данные для обучения обновляются)
DIS_TRAIN_EPOCHS = 1 # количество эпох в одной итерации обучения дискриминатора
ADV_TRAIN_EPOCHS = 25 # количество эпох при обучении генератора на основе соревновательного обучения / обучения с подкреплением
POS_NEG_SAMPLES = 10000

GEN_EMBEDDING_DIM = 150 # ширина слоя Embedding генератора
GEN_HIDDEN_DIM = 150 # количество нейронов в слоях генератора
DIS_EMBEDDING_DIM = 150 # ширина слоя Embedding дискриминатора
DIS_HIDDEN_DIM = 150 # количество нейронов в слоях дискриминатора

# параметры START_LETTER, VOCAB_SIZE, MAX_SEQ_LEN и FILE_PATHS задаются автоматически ниже
START_LETTER = None #! стартовое слово (слово которое подается первым на нейронную сеть генератор)
MAX_SEQ_LEN = None # длина генерируемого примера
VOCAB_SIZE = None # количество слов в словаре
FILE_PATHS = None # пути к файлам набора данных
CLOSING_WORD = None # заключительное слово / слово заполнитель ставящееся в конце предложения

# выбор набора данных и инициализация соответствующих глобальных параметров
if DATASET == 'stihi_ru': # выборка с русскими стихами с сайта stihi_ru (длина примера 10 слов, размер словаря 5 000 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 10
  VOCAB_SIZE = 5000
  CLOSING_WORD = 4999#4997
  FILE_PATHS = {'train': r'datasets/stihi_ru/stihi_ru_realtrain_cotra.txt', 'test': r'datasets/stihi_ru/stihi_ru_realtest_coco.txt',
                'vocab': r'datasets/stihi_ru/stihi_ru_vocab_cotra.pkl', 'saved_models': r'saved_models/stihi_ru'}
elif DATASET == 'coco_image_captions': # выборка COCO Image Captions (длина примера 20 слов, размер словаря 4980 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 20
  VOCAB_SIZE = 4838
  CLOSING_WORD = 1814
  FILE_PATHS = {'train': r'datasets/coco_image_captions/coco_image_captions_train.txt', 'test': r'datasets/coco_image_captions/coco_image_captions_test.txt',
                'vocab': r'datasets/coco_image_captions/coco_image_captions_vocab_cotra_test.pkl', 'saved_models': r'saved_models/coco_image_captions'}

Инициализация генераторов случайных чисел.

In [5]:
# инициализация генераторов случайных чисел.
torch.random.manual_seed(50)
np.random.seed(50)
#системный рандом!!!

Объявление класса Генератора

In [6]:
class Generator(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False):
    super(Generator, self).__init__()
    self.hidden_dim = hidden_dim # количество элементов на скрытом слое
    self.embedding_dim = embedding_dim # размер слоя embedding
    self.max_seq_len = max_seq_len # длина генерируемых примеров
    self.vocab_size = vocab_size # размер словаря использующегося при генерации
    self.gpu = gpu # использование cuda (True/False)
    self.lstm_num_layers = 1 # количество LSTM слоев

    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя embeddings
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.lstm_num_layers) # объявление LSTM слоев
    self.lstm2out = nn.Linear(hidden_dim, vocab_size) # объявление выходного слоя

  def init_hidden(self, batch_size=1):
    # инициализация состояния LSTM слоев
    h = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim)) 
    c = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))

    if self.gpu:
        return h.cuda(), c.cuda()
    else:
        return h, c

  def forward(self, inp, hidden, c):
    """
    Embeds input and applies LSTM one token at a time (seq_len = 1)
    """
    # input dim                                             # batch_size
    emb = self.embeddings(inp)                              # batch_size x embedding_dim
    emb = emb.view(1, -1, self.embedding_dim)               # 1 x batch_size x embedding_dim
    out, (hidden, c) = self.lstm(emb, (hidden, c))                    # 1 x batch_size x hidden_dim (out)
    out = self.lstm2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
    out = F.log_softmax(out, dim=1)
    return out, hidden, c

  def sample(self, num_samples, start_letter=0, degree=1):
    """
    Samples the network and returns num_samples samples of length max_seq_len.

    Outputs: samples, hidden
        - samples: num_samples x max_seq_length (a sampled sequence in each row)
    """

    samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

    h, c = self.init_hidden(num_samples)
    inp = autograd.Variable(torch.LongTensor([start_letter]*num_samples))
    if self.gpu:
        samples = samples.cuda()
        inp = inp.cuda()

    for i in range(self.max_seq_len):
        out, h, c = self.forward(inp, h, c)               # out: num_samples x vocab_size
        out = torch.exp(out)**degree
        out = torch.multinomial(out, 1)  # num_samples x 1 (sampling from each row)
        samples[:, i] = out.view(-1).data

        inp = out.view(-1)

    return samples

  def batchNLLLoss(self, inp, target):
    """
    Returns the NLL Loss for predicting target sequence.

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len

        inp should be target with <s> (start letter) prepended
    """

    loss_fn = nn.NLLLoss()
    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)           # seq_len x batch_size
    target = target.permute(1, 0)     # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        loss += loss_fn(out, target[i])

    return loss     # per batch

  def batchPGLoss(self, inp, target, reward):
    """
    Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
    Inspired by the example in http://karpathy.github.io/2016/05/31/rl/

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len
        - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                  sentence)

        inp should be target with <s> (start letter) prepended
    """

    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)          # seq_len x batch_size
    target = target.permute(1, 0)    # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        # TODO: should h be detached from graph (.detach())?
        for j in range(batch_size):
            loss += -out[j][target.data[i][j]]*reward[j]#     # log(P(y_t|Y_1:Y_{t-1})) * Q

    return loss/batch_size

Объявяление класса Дискриминатора

In [7]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim # размер скрытого слоя
        self.embedding_dim = embedding_dim # ширина слоя embedding
        self.max_seq_len = max_seq_len # длина входного примера
        self.gpu = gpu # использование cuda (True/False)

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя Embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, bidirectional=True, dropout=dropout) # объявление LSTM слоев

        self.lstm2hidden = nn.Linear(2*3*hidden_dim, hidden_dim) # объявление скрытого слоя
        # self.lstm2hidden = nn.Linear(max_seq_len*hidden_dim*2, hidden_dim) # объявление скрытого слоя

        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1) # объявление выходного слоя

    def init_hidden(self, batch_size):
        # инициализация LSTM слоев
        h = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))
        c = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda(), c.cuda()
        else:
            return h, c

    def forward(self, input, hidden, c):
        # input dim                                                # batch_size x seq_len
        # print(input.shape)
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        # print(emb.shape)
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim
        # print(emb.shape)
        out_lstm, (hidden, c) = self.lstm(emb, (hidden, c))                          # 4 x batch_size x hidden_dim

        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        # hidden = out_lstm
        # print(hidden.shape, hidden.view(-1, self.max_seq_len*self.hidden_dim).shape)
        # print(hidden.permute(1, 0, 2).contiguous().shape, hidden.permute(1, 0, 2).contiguous().view(-1, self.max_seq_len*self.hidden_dim*2).shape)

        out = self.lstm2hidden(hidden.view(-1, 6*self.hidden_dim))  # batch_size x 4*hidden_dim
        # out = self.lstm2hidden(hidden.view(-1, self.max_seq_len*self.hidden_dim*2))  # batch_size x 4*hidden_dim
        
        out = torch.relu(out)
        # out = out * torch.sigmoid(0.1 * out) # функция активации Swish: x * sigmoid(b*x)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp):
        """
        Classifies a batch of sequences.

        Inputs: inp
            - inp: batch_size x seq_len

        Returns: out
            - out: batch_size ([0,1] score)
        """

        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return out.view(-1)

    def batchBCELoss(self, inp, target):
        """
        Returns Binary Cross Entropy Loss for discriminator.

         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return loss_fn(out, target)

Объявление функций возвращающих данные

In [8]:
# функция генерации реалиных данных
def sampler_example(batch_size):
  x = data_file_train[np.random.randint(0, len(data_file_train), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

# функция генерации реалиных данных
def sampler_example_test(batch_size):
  x = data_file_test[np.random.randint(0, len(data_file_test), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

Функции обучения генератора

In [9]:
# функция обучения генератора на основе MLE
def train_generator_MLE(gen, gen_opt, real_samples_train, real_samples_test, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        # обучение
        for i in range(0, len(real_samples_train), BATCH_SIZE):
            inp_train, target_train = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp_train, target_train)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(len(real_samples_train) / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_train_NLL = %.4f' % total_loss, end='')

        # тестирование
        test_loss = 0

        for i in range(0, len(real_samples_test), BATCH_SIZE):
            inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                              gpu=CUDA)
            loss = gen.batchNLLLoss(inp_test, target_test)
            test_loss += loss.data.item()
        test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_test_NLL = %.4f' % test_loss)

def test_mle(gen, real_samples_train, real_samples_test):
  '''
  Тестирование генератора на обучающей и тестовой выборках.
  '''
  # тестирование на обучающей
  test_loss = 0
  for i in range(0, len(real_samples_train), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print('average_train_NLL = %.4f' % test_loss, end='')

  # тестирование на тестовой
  test_loss = 0
  for i in range(0, len(real_samples_test), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print(' average_test_NLL = %.4f' % test_loss)

# функция обучения генератора на основе обучения с подкреплением RL
def train_generator_PG(gen, gen_opt, dis, num_batches): # !переименовать!
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*4)        # 64 works best
        inp, target = helpers.prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()
    print()

Функция обучения дискриминатора

In [10]:
# функция обучения дискриминатора
def train_discriminator(discriminator, dis_opt, real_data_samples, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training
    pos_val = real_data_samples[np.random.randint(0, len(real_data_samples), 500)] #sampler_example(250)
    neg_val = generator.sample(500)
    val_inp, val_target = helpers.prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = helpers.batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = helpers.prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        val_pred = discriminator.batchClassify(val_inp)
        print('ДО ОБУЧЕНИЯ: val_acc = %.4f' % (
            torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))

Функция оценки качества генерации текста

In [11]:
# оценка качества по BLEU метрике
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from datetime import datetime
import nltk
import random
from scipy import stats

# функция оценки качества генерации текста по метрике BLEU
def BLEU(reference_sample, test_sample, print_iteration=100, flag_print=True):
  if flag_print:
    print("--- --- ---\nStart BLEU")
  pad = CLOSING_WORD
  #################################################
  reference = []
  for line in reference_sample:
    candidate = []
    for i in line:
      if i == pad:
        break
      candidate.append(i)

    reference.append(candidate)
  #################################################
  hypothesis_list_leakgan = []
  for line in test_sample:
    while line[-1] == str(pad):
      line.remove(str(pad))
    hypothesis_list_leakgan.append(line)
  #################################################
  random.shuffle(hypothesis_list_leakgan)
  #################################################

  smoothing_function = SmoothingFunction().method1

  mass_bleu = []
  for ngram in range(2,6):
      weight = tuple((1. / ngram for _ in range(ngram)))
      bleu_leakgan = []
      bleu_supervise = []
      bleu_base2 = []
      num = 0
      for h in hypothesis_list_leakgan:
          BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference, h, weight, smoothing_function = smoothing_function)
          num += 1
          bleu_leakgan.append(BLEUscore)

          if num%print_iteration == 0 and flag_print:
            print(ngram, num, sum(bleu_leakgan)/len(bleu_leakgan))
          
      mass_bleu.append(1.0 * sum(bleu_leakgan) / len(bleu_leakgan))
      if flag_print:
        print('--- --- ---')
        print(len(weight), '-gram BLEU score : ', 1.0 * sum(bleu_leakgan) / len(bleu_leakgan), "\n")
  return mass_bleu

Функция сохранения моделей

In [12]:
# функция сохранения моделей (сериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer, name):
  state = {
      'default_parameters': {'VOCAB_SIZE': VOCAB_SIZE, 'MAX_SEQ_LEN': MAX_SEQ_LEN, 'GEN_EMBEDDING_DIM': GEN_EMBEDDING_DIM,
                             'GEN_HIDDEN_DIM': GEN_HIDDEN_DIM, 'DIS_EMBEDDING_DIM': DIS_EMBEDDING_DIM, 'DIS_HIDDEN_DIM': DIS_HIDDEN_DIM},
      'data_file_tensor_train': data_file_tensor_train,
      'gen_state_dict': gen.state_dict(),
      'dis_state_dict': dis.state_dict(),
      'gen_optimizer': gen_optimizer.state_dict(),
      'dis_optimizer': dis_optimizer.state_dict(),
  }
  torch.save(state, name)

Функция загрузки моделей

In [13]:
# функция загрузки моделей (десериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def load_models(name):
  if CUDA:
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')

  print('state')
  state = torch.load(name, map_location=device)

  print('default_parameters')
  VOCAB_SIZE = state['default_parameters']['VOCAB_SIZE']
  MAX_SEQ_LEN = state['default_parameters']['MAX_SEQ_LEN']
  GEN_EMBEDDING_DIM = state['default_parameters']['GEN_EMBEDDING_DIM']
  GEN_HIDDEN_DIM = state['default_parameters']['GEN_HIDDEN_DIM']
  DIS_EMBEDDING_DIM = state['default_parameters']['DIS_EMBEDDING_DIM']
  DIS_HIDDEN_DIM = state['default_parameters']['DIS_HIDDEN_DIM']

  print('data_file_tensor_train')
  data_file_tensor_train = torch.tensor(state['data_file_tensor_train'])

  print('Generator')
  gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  gen.load_state_dict(state['gen_state_dict'])
  gen_optimizer = optim.Adam(gen.parameters(), lr=0.001)
  gen_optimizer.load_state_dict(state['gen_optimizer'])

  print('Discriminator')
  dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  dis.load_state_dict(state['dis_state_dict'])
  dis_optimizer = optim.Adagrad(dis.parameters())
  dis_optimizer.load_state_dict(state['dis_optimizer'])

  print('CUDA')
  if CUDA:
    data_file_tensor_train = data_file_tensor_train.cuda()
    gen = gen.cuda()
    dis = dis.cuda()
  
  return [data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
          VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM]

Загрузка набора данных

In [14]:
# загрузка словаря
import pickle

vocab_file = FILE_PATHS['vocab']
word, vocab = pickle.load(open(vocab_file, 'rb'))

In [15]:
# загрузка обучающей выборки
f = open(FILE_PATHS['train'], 'r')
data_file_train = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_train.append(line)
data_file_train = np.array(data_file_train)[:, :MAX_SEQ_LEN]
print("Примеров в обучающей выборке: ", len(data_file_train))

Примеров в обучающей выборке:  80000


In [16]:
# загрузка тестовой выборки
f = open(FILE_PATHS['test'], 'r')
data_file_test = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_test.append(line)
data_file_test = np.array(data_file_test)[:, :MAX_SEQ_LEN]
print("Примеров в тестовой выборке: ", len(data_file_test))

Примеров в тестовой выборке:  5000


In [17]:
# примеры из обучающей выборки
print("Примеры из обучающей выборки")
samples = sampler_example(50)[0]
output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tПример: ", output)    

Примеры из обучающей выборки
# 0 	Пример:  A street in front of an airport with people going in the entrance .            
# 1 	Пример:  Blue and green bird sitting on the ground next to plants .                
# 2 	Пример:  A racer is sitting on a horse in the gates .                  
# 3 	Пример:  A man skiing a jump , while in the background there is snow melting off the slopes .    
# 4 	Пример:  A man that is holding a knife and a pot with broccoli .              
# 5 	Пример:  A bed in a bedroom that has lots of things sitting on top of it .        
# 6 	Пример:  A skateboarder does a handstand trick on a skate - ramp .                
# 7 	Пример:  A tall white building with three vehicles parked in front of it .              
# 8 	Пример:  A man admiring a black and yellow motorcycle on a white background .              
# 9 	Пример:  A cup of soup , a sandwich and a sweet roll on a table .          
# 10 	Пример:  Traffic on a busy city road with street lights and Christmas 

Оценка качества примеров из обучающей выборки на основе BLEU  (для сравнения с генератором)

In [18]:
%%time
# оценка качества на основе примеров из обучающей выборки (для сравнения с генератором)
print("Оценка примеров из обучающей выборки на основе BLEU")
BLEU(data_file_test.tolist(), data_file_train[:500].tolist(), print_iteration=100)

'''
BLEU примеров из обучающей выборки
--- --- ---
Start BLEU
2 100 0.5365504958995001
2 200 0.5426700180034696
2 300 0.5425210147596119
2 400 0.5438925313146432
2 500 0.5499117864293879
--- --- ---
2 -gram BLEU score :  0.5499117864293879
3 100 0.4252853298296578
3 200 0.4294441261529973
3 300 0.42938411299670276
3 400 0.432946142036555
3 500 0.436991427933691
--- --- ---
3 -gram BLEU score :  0.436991427933691
4 100 0.3118228221158724
4 200 0.31347352868277367
4 300 0.3133847969805193
4 400 0.3148005592801212
4 500 0.3173224414085976
--- --- ---
4 -gram BLEU score :  0.3173224414085976
5 100 0.20748007111715946
5 200 0.21057918753505803
5 300 0.20884027925072032
5 400 0.20986772415037908
5 500 0.21083373415979106
--- --- ---
5 -gram BLEU score :  0.21083373415979106
'''

Оценка примеров из обучающей выборки на основе BLEU
--- --- ---
Start BLEU
2 100 0.5570250679278492
2 200 0.556688037824323
2 300 0.5493089304838155
2 400 0.5494658337492588
2 500 0.5499117864293878
--- --- ---
2 -gram BLEU score :  0.5499117864293878 

3 100 0.44196089553285683
3 200 0.447768125311308
3 300 0.43858963216173663
3 400 0.4372307787789591
3 500 0.43699142793369133
--- --- ---
3 -gram BLEU score :  0.43699142793369133 

4 100 0.31414379080893645
4 200 0.3275024883519048
4 300 0.3213421972239594
4 400 0.3174880435602007
4 500 0.31732244140859767
--- --- ---
4 -gram BLEU score :  0.31732244140859767 

5 100 0.21122823761654302
5 200 0.21973281984255646
5 300 0.21605178827556978
5 400 0.21085774538348726
5 500 0.2108337341597913
--- --- ---
5 -gram BLEU score :  0.2108337341597913 

CPU times: user 7min 58s, sys: 52.6 ms, total: 7min 58s
Wall time: 7min 58s


Создание нейронных сетей генератора и дискриминатора

In [19]:
# объявление нейронных сетей генератора и дискриминатора, подготовка выборок данных для использования pytorch
gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)

if CUDA:
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()
else:
  data_file_tensor_train = torch.tensor(data_file_train)
  data_file_tensor_test = torch.tensor(data_file_test)

gen_optimizer = optim.Adam(gen.parameters(), lr=0.001) #, lr=0.001
dis_optimizer = optim.Adagrad(dis.parameters())

Обучение генератора на основе MLE

In [20]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 8.5446 average_test_NLL = 8.5451
CPU times: user 1.3 s, sys: 397 ms, total: 1.7 s
Wall time: 1.8 s


In [21]:
%%time
# обучение генератора на основе MLE / предобучение генератора
print('Запуск обучения генератора на основе MLE...')
gen_optimizer = optim.Adam(gen.parameters())#, lr=0.0002
train_generator_MLE(gen, gen_optimizer, data_file_tensor_train, data_file_tensor_test, MLE_TRAIN_EPOCHS) # MLE_TRAIN_EPOCHS

test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_mle.pytorch')
# epoch 1 : .......... average_train_NLL = 1.8447 average_test_NLL = 1.9937

Запуск обучения генератора на основе MLE...
epoch 1 : .......... average_train_NLL = 4.5138 average_test_NLL = 3.3173
epoch 2 : .......... average_train_NLL = 3.0475 average_test_NLL = 2.9099
epoch 3 : .......... average_train_NLL = 2.7634 average_test_NLL = 2.7012
epoch 4 : .......... average_train_NLL = 2.5905 average_test_NLL = 2.5820
epoch 5 : .......... average_train_NLL = 2.4759 average_test_NLL = 2.4898
epoch 6 : .......... average_train_NLL = 2.3922 average_test_NLL = 2.4348
epoch 7 : .......... average_train_NLL = 2.3276 average_test_NLL = 2.3860
epoch 8 : .......... average_train_NLL = 2.2767 average_test_NLL = 2.3403
epoch 9 : .......... average_train_NLL = 2.2326 average_test_NLL = 2.3062
epoch 10 : .......... average_train_NLL = 2.1932 average_test_NLL = 2.2736
epoch 11 : .......... average_train_NLL = 2.1576 average_test_NLL = 2.2472
epoch 12 : .......... average_train_NLL = 2.1273 average_test_NLL = 2.2134
epoch 13 : .......... average_train_NLL = 2.0967 average_test_NLL

In [22]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 1.8009 average_test_NLL = 1.9609
CPU times: user 1.41 s, sys: 327 ms, total: 1.74 s
Wall time: 1.75 s


Генерация примеров текстов на основе MLE

In [23]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1
# 0 	Пример:  The train traveling down the track on town track just set out of the park .                           	Оценка:  [0.5803810000880093, 0.4539847774172143, 0.1531682455208201, 0.08078132896612913]
# 1 	Пример:  The fence sitting inside the bathroom surrounded by trees and tube .                                  	Оценка:  [0.41675437673324534, 0.268229131353767, 0.10322080150552766, 0.05891010340986106]
# 2 	Пример:  A train moving tracks next to a wooden size platform in the country in a dimly environment .          	Оценка:  [0.6529286250990105, 0.4558685532582043, 0.2732231510487644, 0.12834826432879642]
# 3 	Пример:  A woman on the tennis court preparing to hit with a bat .                                             	Оценка:  [0.4893604849295929, 0.34174500988297, 0.22012248449054683, 0.1079711005739161]
# 4 	Пример:  A cat is sitting on a train along with many other riders ' s figurine .                               

Оценка качества генерации текста после обучения с помощью MLE

In [24]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.5297461279630729
2 200 0.5355221356420088
2 300 0.5266944489877361
2 400 0.5225363400622878
2 500 0.5234044964177561
--- --- ---
2 -gram BLEU score :  0.5234044964177561 

3 100 0.3926205639252202
3 200 0.3957693278204802
3 300 0.38304084088080326
3 400 0.3821505535401551
3 500 0.38369548201661996
--- --- ---
3 -gram BLEU score :  0.38369548201661996 

4 100 0.25846203230599557
4 200 0.2598156251690873
4 300 0.2497098871280157
4 400 0.24866964006164907
4 500 0.24843516011889202
--- --- ---
4 -gram BLEU score :  0.24843516011889202 

5 100 0.1546238787311136
5 200 0.15696177542613282
5 300 0.1508937330373346
5 400 0.1498088051055619
5 500 0.14932465150034782
--- --- ---
5 -gram BLEU score :  0.14932465150034782 

CPU times: user 8min, sys: 175 ms, total: 8min
Wall time: 8min


Оценка с возведением в степень

In [25]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1.5
# 0 	Пример:  The man is doing the trick on a court near a man .                                                    	Оценка:  [0.5548826333938605, 0.37160681631842096, 0.13181065657459776, 0.07163684030886805]
# 1 	Пример:  A man holding up a donut in a bowl of flowers in a room .                                             	Оценка:  [0.716350399411379, 0.6109741448719966, 0.404727200247809, 0.17575428316242378]
# 2 	Пример:  A woman in a dress eating a pizza with a slice missing .                                              	Оценка:  [0.5548826333938605, 0.4090062242255915, 0.2518750835198491, 0.12026082240959093]
# 3 	Пример:  A green train down train station on the tracks near a mountain .                                      	Оценка:  [0.5231483637805969, 0.3572998474390311, 0.22759483012793272, 0.11089345578518284]
# 4 	Пример:  A lady is holding a tennis racket on a clay court .                                               

In [26]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.563796489069254
2 200 0.5639550303900202
2 300 0.5637841267854927
2 400 0.5605215308862388
2 500 0.5601664492111114
--- --- ---
2 -gram BLEU score :  0.5601664492111114 

3 100 0.47213855104687286
3 200 0.47811324904365854
3 300 0.4774650667857076
3 400 0.4719277409628813
3 500 0.47195595945252516
--- --- ---
3 -gram BLEU score :  0.47195595945252516 

4 100 0.3701483948064162
4 200 0.3755901049715418
4 300 0.37449450546496804
4 400 0.3669356998589144
4 500 0.3652760221418803
--- --- ---
4 -gram BLEU score :  0.3652760221418803 

5 100 0.26310827987968355
5 200 0.2629093684592063
5 300 0.263469259148831
5 400 0.2551864176682963
5 500 0.2528469331763989
--- --- ---
5 -gram BLEU score :  0.2528469331763989 

CPU times: user 7min 40s, sys: 170 ms, total: 7min 40s
Wall time: 7min 40s


Предварительное обучение дискриминатора

In [27]:
%%time
# предобучение дискриминатора
print('Запуск обучения дискриминатора...')
# dis_optimizer = optim.Adagrad(dis.parameters()) # , lr=0.0001
# dis_optimizer = optim.Adam(gen.parameters(), lr=0.001, weight_decay=1e-5)#0.001
# dis_optimizer = optim.Adadelta(gen.parameters())
# dis_optimizer = optim.Adagrad(dis.parameters())#, lr=0.0001)#, weight_decay=1e-5)
#, weight_decay=1e-5) # регуляризация
train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, DIS_TRAIN_ITERATIONS, DIS_TRAIN_EPOCHS)#25, 1 | (15, 3), (25, 1)

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_pretraining_dis.pytorch')

# ДО ОБУЧЕНИЯ: val_acc = 0.5230
# d-step 50 epoch 1 : .......... average_loss = 0.2859, train_acc = 0.9136, val_acc = 0.5240

Запуск обучения дискриминатора...
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 1 epoch 1 : .......... average_loss = 0.4372, train_acc = 0.8496, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 2 epoch 1 : .......... average_loss = 0.3523, train_acc = 0.8874, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 3 epoch 1 : .......... average_loss = 0.3495, train_acc = 0.8867, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 4 epoch 1 : .......... average_loss = 0.3427, train_acc = 0.8874, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 5 epoch 1 : .......... average_loss = 0.3434, train_acc = 0.8885, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 6 epoch 1 : .......... average_loss = 0.3310, train_acc = 0.8898, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 7 epoch 1 : .......... average_loss = 0.3262, train_acc = 0.8888, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 8 epoch 1 : .......... average_loss = 0.3274, train_acc = 0.8879, val_acc = 0.5000
ДО ОБУ

Состязательное обучение. Обучение генератора на основе обучения с подкреплением

In [28]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.7516 average_test_NLL = 1.9832

average_train_NLL = 1.8009 average_test_NLL = 1.9609


In [29]:
%%time
# gen_optimizer = optim.Adagrad(gen.parameters(), lr=0.0005)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters())
# gen_optimizer = optim.Adam(gen.parameters(), lr=0.0001)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters(), lr=0.001)#, lr=0.002

# состязательное обучение генератора
print('\nStarting Adversarial Training...')

for epoch in range(ADV_TRAIN_EPOCHS):# ADV_TRAIN_EPOCHS
    print('\n--------\nEPOCH %d\n--------' % (epoch+1))
    # обучение генератора
    print('\nAdversarial Training Generator : ', end='')
    train_generator_PG(gen, gen_optimizer, dis, 1)#

    # тестирование nll
    print('\nTesting Generator : ', end='')
    test_mle(gen, data_file_tensor_train, data_file_tensor_test)

    # обучение дискриминатора
    print('\nAdversarial Training Discriminator : ')
    train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, 5, 1)#3, 1

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')
# epoch 3 : .......... average_train_NLL = 1.8005 average_test_NLL = 1.9824
# average_train_NLL = 1.7340 average_test_NLL = 2.0599


Starting Adversarial Training...

--------
EPOCH 1
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 1.8014 average_test_NLL = 1.9606

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val_acc = 0.8010
d-step 1 epoch 1 : .......... average_loss = 0.1623, train_acc = 0.9534, val_acc = 0.8070
ДО ОБУЧЕНИЯ: val_acc = 0.8120
d-step 2 epoch 1 : .......... average_loss = 0.1551, train_acc = 0.9574, val_acc = 0.7940
ДО ОБУЧЕНИЯ: val_acc = 0.7980
d-step 3 epoch 1 : .......... average_loss = 0.1643, train_acc = 0.9532, val_acc = 0.8080
ДО ОБУЧЕНИЯ: val_acc = 0.8000
d-step 4 epoch 1 : .......... average_loss = 0.1640, train_acc = 0.9532, val_acc = 0.8010
ДО ОБУЧЕНИЯ: val_acc = 0.7960
d-step 5 epoch 1 : .......... average_loss = 0.1642, train_acc = 0.9534, val_acc = 0.8040

--------
EPOCH 2
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 1.8021 average_test_NLL = 1.9603

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val

Тестирование генератора на обучающей выборке после состязательного обучения.

In [30]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.5186 average_test_NLL = 2.0730
# average_train_NLL = 1.4459 average_test_NLL = 1.9926

average_train_NLL = 1.8011 average_test_NLL = 1.9544
CPU times: user 1.41 s, sys: 395 ms, total: 1.81 s
Wall time: 1.81 s


Генерация примеров текстов на основе SeqGAN

In [31]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе SeqGAN
Degree: 1
# 0 	Пример:  A very pretty blue flowered bus waiting for the city .                                                	Оценка:  [0.39735970711951313, 0.29744417462950146, 0.11154288433080788, 0.06268007390554406]
# 1 	Пример:  a Children are playing a video game on a sink that is sitting next to construction .                  	Оценка:  [0.7015021476502369, 0.47820615913318404, 0.28320384389628483, 0.13208555288986817]
# 2 	Пример:  This plate contains meat sandwich and silverware is sitting and a pot in the water .                  	Оценка:  [0.6155870112510925, 0.4721631957086835, 0.28051550320162943, 0.13108152886996202]
# 3 	Пример:  A computer monitor is with pillows , a checkered box .                                                	Оценка:  [0.3627381250550058, 0.19407576238520263, 0.08097785064266204, 0.048514214015354815]
# 4 	Пример:  A kitchen that close up oven inside of a flat wall .                                     

Оценка качества генерации на основе SeqGAN

In [32]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 1
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
Degree: 1
--- --- ---
Start BLEU
2 100 0.5074841333882112
2 200 0.514819199850076
2 300 0.5170806356845506
2 400 0.5191300973781708
2 500 0.5220390101251234
--- --- ---
2 -gram BLEU score :  0.5220390101251234 

3 100 0.38127956558435677
3 200 0.38821018244951083
3 300 0.38787361175157303
3 400 0.3886326887782336
3 500 0.3928964532399967
--- --- ---
3 -gram BLEU score :  0.3928964532399967 

4 100 0.2495628351282375
4 200 0.2609661607096216
4 300 0.260242412511415
4 400 0.26050172139711614
4 500 0.26614798307671744
--- --- ---
4 -gram BLEU score :  0.26614798307671744 

5 100 0.15493427465216825
5 200 0.16303737643799224
5 300 0.16267665562072717
5 400 0.16399300449864895
5 500 0.17012601345146197
--- --- ---
5 -gram BLEU score :  0.17012601345146197 

CPU times: user 7min 52s, sys: 176 ms, total: 7min 53s
Wall time: 7min 52s


Оценка с возведением в степень

In [33]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN с возведением в степень")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе SeqGAN с возведением в степень
Degree: 1.5
# 0 	Пример:  A man standing in a room with a table in front of a building .                                        	Оценка:  [0.7433919416750281, 0.716882776507279, 0.6452772832060505, 0.5581707898352563]
# 1 	Пример:  A person in a room with a white laptop computer .                                                     	Оценка:  [0.5104177855340404, 0.46618580376022084, 0.39293620890698494, 0.2720405498897358]
# 2 	Пример:  A man on a table with a man standing on the ground .                                                  	Оценка:  [0.6407232755171874, 0.6306964608093198, 0.5454951299940093, 0.35367853716020087]
# 3 	Пример:  A person standing on top of a small grass field .                                                     	Оценка:  [0.5104177855340404, 0.46618580376022084, 0.3656683453259979, 0.2568300857166973]
# 4 	Пример:  A couple of men standing in front of a table with a laptop .             

In [34]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
--- --- ---
Start BLEU
2 100 0.5746959166408108
2 200 0.5712008863573584
2 300 0.567327534197651
2 400 0.569402426596509
2 500 0.5716843921414854
--- --- ---
2 -gram BLEU score :  0.5716843921414854 

3 100 0.5050846828032072
3 200 0.49620438762803126
3 300 0.49256672478599955
3 400 0.4990660653296927
3 500 0.5028216465603539
--- --- ---
3 -gram BLEU score :  0.5028216465603539 

4 100 0.4125649208522431
4 200 0.4031958117320594
4 300 0.4005020051371067
4 400 0.4098193662865048
5 100 0.30949307637919304
5 200 0.2988029014742637
5 300 0.2952895890220848
5 400 0.3042843311354224
5 500 0.3068178815954311
--- --- ---
5 -gram BLEU score :  0.3068178815954311 

CPU times: user 7min 41s, sys: 165 ms, total: 7min 42s
Wall time: 7min 41s


Загрузка сохраненной модели

In [35]:
# загрузка моделей
[data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM,
 DIS_HIDDEN_DIM] = load_models(FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')#[seqgan_mle, seqgan_pretraining_dis, seqgan_adversarial_training]

if(CUDA):
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_tensor_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()

state
default_parameters
data_file_tensor_train
Generator
Discriminator
CUDA


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Оценка разнообразия текста

In [36]:
degree = 1.5

In [37]:
# плагиат из обучающей выборки
samples = gen.sample(1500, degree=degree).cpu().detach().numpy().tolist()
# samples = data_file_test.tolist()[:500]
train_samples = data_file_train.tolist()
n = 0
for i in range(len(samples)):
  if samples[i] in train_samples:
    n += 1
  if i%(len(samples)//10) == 0:
    print(i/len(samples)*100, "%")
print("Совпало сгенерированных примеров с обучающей выборкой: ", n, "из", len(samples))
print("Плагиат: ", n/len(samples)*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Совпало сгенерированных примеров с обучающей выборкой:  9 из 1500
Плагиат:  0.6 %


In [38]:
# оригинальность примеров
N_samples = 1500
samples_1 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()
samples_2 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

n = 0
for i in range(len(samples_1)):
  if samples_1[i] in samples_2:
    n += 1
  if i%(len(samples_1)//10) == 0:
    print(i/len(samples_1)*100, "%")

print("Оригинальность генерируемых примеров")
print("Совпало примеров из выборки 1 с примерами из выборки 2: ", n, "из", len(samples_1))
print("Плагиат: ", n/len(samples_1)*100, "%")
print("Оригинальность: ", (1-n/len(samples_1))*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Оригинальность генерируемых примеров
Совпало примеров из выборки 1 с примерами из выборки 2:  22 из 1500
Плагиат:  1.4666666666666666 %
Оригинальность:  98.53333333333333 %


In [39]:
# оригинальность примеров подход 2
N_samples = 1500
samples = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

i = 0
n = 0
while len(samples)>0:
  if samples[0] not in samples[1:]:
    n += 1
  if i%(N_samples//10) == 0:
    print(i/N_samples*100, "%")
  del(samples[0])
  i += 1

print("Оригинальность генерируемых примеров")
print("Сгенерировано оригинальных примеров: ", n, "из", N_samples)
print("Оригинальность: ", n/N_samples*100, "%")
print("Плагиат: ", (1-n/N_samples)*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Оригинальность генерируемых примеров
Сгенерировано оригинальных примеров:  1493 из 1500
Оригинальность:  99.53333333333333 %
Плагиат:  0.4666666666666708 %
